<h2> Semi-automatic literature search </h2> 

In [1]:
# import internal .py modules
import file_path_management as fpath
import public_library as plib

2023-08-01 10:02:40 Didis-MacBook-Pro.local metapub.config[8654] WARNING NCBI_API_KEY was not set.


In [2]:
# import packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
import re
import time
import numpy as np
import numpy as np

<h3> Parameters: </h3>

In [3]:
# searching keywords lexicon
# search in all fields
# "" means exact match, otherwise the search engine will treat every word separately

# search_kws_lexicon = (macaque OR macaca OR "rhesus monkey") AND (thalamus OR thalamic OR thalamocortical OR "thalamo-cortical")

# academic databases
# Google Scholar: "https://scholar.google.com/"
# (macaque OR macaca OR "rhesus monkey") (thalamus OR thalamic OR thalamocortical OR "thalamo-cortical")
# https://scholar.google.com/scholar?start=0&q=(macaque+OR+macaca+OR+%22rhesus+monkey%22)+(thalamus+OR+thalamic+OR+thalamocortical+OR+%22thalamo-cortical%22)&hl=en&as_sdt=0,5
# 31.07.2023
# 131000 results
# get 1000 results in practice

# Web of Science: "https://www.webofscience.com/wos/woscc/advanced-search" # can be exported to excel file
# (ALL=(macaque) OR ALL=(macaca) OR All=("rhesus monkey")) AND (ALL=(thalamus) OR ALL=(thalamic) OR ALL=(thalamocortical) OR ALL=("thalamo-cortical"))
# https://www.webofscience.com/wos/woscc/summary/cbcda45c-f1a5-45d2-bc24-ff8c17e0c083-9a6726cd/relevance/1
# 27.07.2023
# 1976 results

# PubMed: "https://pubmed.ncbi.nlm.nih.gov/advanced/" # can be exported to .csv file
# (macaque OR macaca OR "rhesus monkey") AND (thalamus OR thalamic OR thalamocortical OR "thalamo-cortical")
# https://pubmed.ncbi.nlm.nih.gov/?term=(macaque%20OR%20macaca%20OR%20%22rhesus%20monkey%22)%20AND%20(thalamus%20OR%20thalamic%20OR%20thalamocortical%20OR%20%22thalamo-cortical%22)&page=1
# 27.07.2023
# 2606 results

# Europe PMC = "https://europepmc.org/advancesearch" # search resuts can be exported to .csv file
# (macaque OR macaca OR "rhesus monkey") AND (thalamus OR thalamic OR thalamocortical OR "thalamo-cortical") AND (LANG:"eng" OR LANG:"en" OR LANG:"us")
# https://europepmc.org/search?query=%28macaque%20OR%20macaca%20OR%20%22rhesus%20monkey%22%29%20AND%20%28thalamus%20OR%20thalamic%20OR%20thalamocortical%20OR%20%22thalamo-cortical%22%29%20AND%20%28LANG%3A%22eng%22%20OR%20LANG%3A%22en%22%20OR%20LANG%3A%22us%22%29&page=1
# 27.07.2023
# 9139 results

acad_dbs = ["Google Scholar", "Web of Science", "PubMed", "Europe PMC"]

# initial urls for specified searching keyword lexicon and all academic databases
init_urls = {
    "gs": "https://scholar.google.com/scholar?start=0&q=(macaque+OR+macaca+OR+%22rhesus+monkey%22)+(thalamus+OR+thalamic+OR+thalamocortical+OR+%22thalamo-cortical%22)&hl=en&as_sdt=0,5",
    "wos": "https://www.webofscience.com/wos/woscc/summary/cbcda45c-f1a5-45d2-bc24-ff8c17e0c083-9a6726cd/relevance/1",
    "pubmed": "https://pubmed.ncbi.nlm.nih.gov/?term=(macaque%20OR%20macaca%20OR%20%22rhesus%20monkey%22)%20AND%20(thalamus%20OR%20thalamic%20OR%20thalamocortical%20OR%20%22thalamo-cortical%22)&page=1",
    "eupmc": "https://europepmc.org/search?query=%28macaque%20OR%20macaca%20OR%20%22rhesus%20monkey%22%29%20AND%20%28thalamus%20OR%20thalamic%20OR%20thalamocortical%20OR%20%22thalamo-cortical%22%29%20AND%20%28LANG%3A%22eng%22%20OR%20LANG%3A%22en%22%20OR%20LANG%3A%22us%22%29&page=1"
}

# seed literature list
seed_litera_list = []

# cocomac literature list
cocomac_litera_list = []

<h3> Predefined fucntions: </h3> 

<h5> Method 1: Searching academic databases: Web of Science, PubMed, Europe PMC, Google Scholar </h5> 

In [4]:
def search_webofscience(init_url):
    try:
        print("Searching Web of Science...")
        # search on the website and export the search results
        return True
    except:
        return False

In [5]:
def search_pubmed(init_url):
    try:
        print("Searching PubMed...")
        # search on the website and export the search results
        return True
    except:
        return False

In [6]:
def search_eupmc(init_url):
    try:
        print("Searching Europe PMC...")
        # search on the website and export the search results
        return True
    except:
        return False

In [7]:
def search_google_scholar(first_page, start, end):
    print("Searching Google Scholar...")
    
    # # request the first page 1
    # proxies = plib.get_proxies()
    # soup = plib.request_webpage(first_page, proxies)
    # # print(soup)

    # # print the number of search results
    # num_results_str_list = soup.find_all("div", {"class": "gs_ab_mdw"})
    # for item in num_results_str_list:
    #     num_results_str = []
    #     if "results" in item.get_text():
    #         num_results_str = item.get_text().split()
    # num_results = num_results_str[1]
    # num_results = int(re.sub(r"[^0-9]", "", num_results))
    # pages = int(num_results/10)
    # print("Google Scholar searched " + str(num_results) + " results" + " displayed in " + str(pages) + " pages.")
    
    # iterate all pages and record the results
    for page in range(start, end):
        # sleep
        time.sleep(random.randint(3, 5))
        # if(page%20 == 0):
        #     time.sleep(random.randint(5, 10)*60)
        
        # search a page
        start = page * 10
        page_url = first_page.split("?start=")[0] + "?start=" + str(start) + "&q=" + first_page.split("&q=")[1]
        # print(page_url)
        proxies = plib.get_proxies()
        soup = plib.request_webpage(page_url, proxies)
        # print(soup)
        items = soup.select('[data-lid]')
        # print(items)

        for item in items:
            # columns = ["title", "url", "url_type", "full_text_url", "full_text_type", "full_text_source"]
            # get title
            try:
                title = item.select("h3")[0].select("a")[0].get_text().strip()
            except:
                title = np.nan
            # print(add_title)

            # get url
            try:
                url = item.select("h3")[0].select("a", href = True)[0]["href"]
            except:
                url = np.nan
            # print(url)

            # get url_type
            try:
                url_type = item.select("h3")[0].select("span", {"class": "gs_ct"})[0].get_text().strip()
            except:
                url_type = np.nan

            # get full_text_url
            try:
                add_full_text_link = item.find_all("div", {'class': "gs_or_ggsm"})[0].find_all("a", href = True)[0]["href"]
            except:
                add_full_text_link = np.nan
            # print(add_full_text_link)

            # get full_text_type, full_text_source
            try:
                full_text_type = item.find_all("div", {'class': "gs_or_ggsm"})[0].find_all("a", href = True)[0].get_text().strip().split()[0]
                full_text_source = item.find_all("div", {'class': "gs_or_ggsm"})[0].find_all("a", href = True)[0].get_text().strip().split()[1]
            except:
                full_text_type = np.nan
                full_text_source = np.nan
            # print(full_text_type)

            # print(full_text_source)
            row = {
                "title": [title],
                "url": [url],
                "url_type": [url_type],
                "full_text_url": [add_full_text_link],
                "full_text_type": [full_text_type],
                "full_text_source": [full_text_source]
            }
            # print(row)
            columns = ["title", "url", "url_type", "full_text_url", "full_text_type", "full_text_source"]
            if plib.add_row_to_csv(fpath.poten_litera_gs, row, columns):
                None
            else:
                print("Error detected when adding a row to csv!")
        print("Searching page ", page, "succeded!")
# --------------------start of test code--------------------
# init_url = init_urls["gs"]
# search_google_scholar(init_url, 0, 2)
# ---------------------end of test code---------------------

<h5> Method 2: Spanning citations of seed papers </h5> 

In [8]:
def span_citations_of_seed_papers(seed_paper_list, num_span_times):
    print("Spanning citations of seed papers...")
    try:
        return True
    except:
        return False

<h5> Method 3: Searching existing connection databse CoCoMac </h5> 

In [9]:
def search_cocomac(cocomac_paper_list):
    print("Searching CoCoMac database...")
    try:    
        return True
    except:
        return False

<h3> Main program: </h3> 

In [10]:
# first we need to search all related literature that might include data or information of thalamocortical connections
# search for potentially related literature using the following listed 3 methods

<h5> Method 1: Searching academic databases: Web of Science, PubMed, Europe PMC, Google Scholar </h5> 

In [11]:
# method 1: search acdemic databases using keywords
if search_webofscience(init_urls["wos"]):
    print("Searching Web of Science succeeded!")
else:
    print("Attention! Something went wrong when searching Web of Science!")

if search_pubmed(init_urls["pubmed"]):
    print("Searching PubMed succeeded!")
else:
    print("Attention! Something went wrong when searching PubMed!")

if search_eupmc(init_urls["eupmc"]):
    print("Searching Europe PMC succeeded!")
else:
    print("Attention! Something went wrong when searching Europe PMC!")

Searching Web of Science...
Searching Web of Science succeeded!
Searching PubMed...
Searching PubMed succeeded!
Searching Europe PMC...
Searching Europe PMC succeeded!


In [12]:
f = open(fpath.poten_litera_gs, "w")
f.truncate()
f.close()

search_google_scholar(init_urls["gs"], 0, 100)

Searching Google Scholar...
Searching page  0 succeded!
Searching page  1 succeded!
Searching page  2 succeded!
Searching page  3 succeded!
Searching page  4 succeded!
Searching page  5 succeded!
Searching page  6 succeded!
Searching page  7 succeded!
Searching page  8 succeded!
Searching page  9 succeded!
Searching page  10 succeded!
Searching page  11 succeded!
Searching page  12 succeded!
Searching page  13 succeded!
Searching page  14 succeded!
Searching page  15 succeded!
Searching page  16 succeded!
Searching page  17 succeded!
Searching page  18 succeded!
Searching page  19 succeded!
Searching page  20 succeded!
Searching page  21 succeded!
Searching page  22 succeded!
Searching page  23 succeded!
Searching page  24 succeded!
Searching page  25 succeded!
Searching page  26 succeded!
Searching page  27 succeded!
Searching page  28 succeded!
Searching page  29 succeded!
Searching page  30 succeded!
Searching page  31 succeded!
Searching page  32 succeded!
Searching page  33 succed

<h5> Method 2: Spanning citations of seed papers </h5> 

In [13]:
# # method 2: spanning citations of seed papers
# if span_citations_of_seed_papers(fpath.seed_paper_list, num_span_times):
#     print("Spanning citations of seed literature list succeeded!")
# else:
#     print("Attention! Something went wrong when spanning citations of seed literature list!")

<h5> Method 3: Searching existing connection databse CoCoMac </h5> 

In [14]:
# # method 3: search existing connectome databases
# if search_cocomac(cocomac_paper_list):
#     print("Searching connectome databases succeeded!")
# else:
#     print("Attention! Something went wrong when searching connectome databases!")

<h3> Next step: searched literature data preprocessing </h3>